# Differences Between Bias Instability Models

**Goal:  Determine which bias instability model best simulates bias instability.** 

We are interested in comparing three different models:
- First Order Markov 
- MATLAB Model (or Filter Model)
- Aceinna Model

## First Order Markov
We first saw the first order markov model in K. Jerath's Presentation *Noise Modeling of Sensors:  The Allan Variance Method*.

Each point $b$ in the simulated time series is calculated from an initial conditon $b_{0}$ according to $\dot{b} = -\frac{b}{T_{c}}+\eta_{b}$. 

The appeal of the first order markov model is that **correlation time $T_{c}$** is explicitly defined in the model.

## MATALB Model (or Filter Model)
MATLAB implements bias instability through its `imuSensor` object. This model filters a white noise time series into 1/f noise - also known as bias instability.

MATLAB is closed source, so it's not possible (right now) to inspect the code directly. However, K. Jerath implemented the same method in **[1](https://www.mathworks.com/matlabcentral/fileexchange/64590-allan-variance-analysis-of-simulated-sensor-noise)**.

The drawback of this model is that correlation time is not explicitly used. However, the model consistently yields a PSD with the correct characteristics and can be used to accurately determine the bias instability coefficient from an Allan deviation plot. 

## Aceinna Model
This model is an implementation of First Order Gauss-Markov model (discussed above). The exact equation used is derived in Section 3.2 of [Quinchia, et. al](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3812568/).

This model computes a discrete bias instability time series according to:

$x_{k} = (1-\beta\Delta t)x_{k-1} + \sigma_{GM}\sqrt{(1-e^{\frac{-2\Delta t}{T_{c}}})}w_{k}$

We can rewrite this equation as:

$x_{k}=(1-\frac{1}{T_{c}}\frac{1}{f_{s}})x_{k-1} + B \sqrt{(1-e^{\frac{-2}{f_{s}T_{c}}})}w_{k}$

Then, it is natural to provide a bias coefficient $B$, a correlation time $T_{c}$, and sampling frequency $f_{s}$ to compute our bias instability time series. 

Note that $w{k}$ are samples from a standard normal distribution.

The Acienna model is explicitly implemented for 3 axis. We will have to be aware of this fact in the experiments below.

---

# References

**[1]**  Kshitij Jerath (2021). Allan variance analysis of simulated sensor noise (https://www.mathworks.com/matlabcentral/fileexchange/64590-allan-variance-analysis-of-simulated-sensor-noise), MATLAB Central File Exchange. Retrieved July 27, 2021. 

# Imports

In [3]:
# Clone Allan Online repo to notebook directory
!git clone https://github.com/nurriol2/allan-online.git

# Clone GNSS INS Simulation repo to notebook directory 
!git clone https://github.com/Aceinna/gnss-ins-sim.git

from IPython.display import clear_output
clear_output()

In [8]:
# Make Allan Online files `import`-able
allan_online_dir = "./allan-online"
sys.path.append(os.path.abspath(allan_online_dir))

# Make Aceinna GNSS INS Simulation files `import`-able
aceinna_pathgen_dir = "./gnss-ins-sim/gnss_ins_sim/pathgen" # Only need `gnss-ins-sim/gnss_ins_sim/pathgen/pathgen.py`
sys.path.append(os.path.abspath(aceinna_pathgen_dir))


# Standard imports
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import welch
import sys
import os


# Function for calculating the overlapping Allan deviation of a time series
from allan_variance import overlapping_allan_deviation
# First Order Markov model for bias instability
from noise_synthesis import make_bias_instability_series as first_order_markov
# MATLAB Model - Shaping white noise with a filter
from noise_synthesis import simulate_flicker_noise as matlab_model
# Acienna bias instability model
from pathgen import bias_drift as aceinna_model